### Libraries

In [1700]:
from pathlib import Path
from pprint import pprint

### Variables

In [1701]:
edl_path = 'reference\z old\Avid-Sequence.edl'
drop_frame = False
video_edits = []
audio_edits = []
# edit_files = []

### Open File

In [1702]:
# Opens EDL and extracts its contents into a single string.
def open_edl(edl_path):
    with open(edl_path, 'r') as edl_file:
        edl_contents = edl_file.read()
    edl_extract(edl_contents)    

In [1703]:
# Breasks EDL into a list of fields for each line
def edl_extract(edl_contents): 
    edl_lines = edl_contents.splitlines()

    edl_extracted = []

    for line in edl_lines:
        columns = line.split()
        edl_extracted.append(columns)
    
    
    simplify_edl(edl_extracted)


### Simplify EDL

In [1704]:
def simplify_edl(edl):
	detect_dropframe(edl)
	simplified_edl = []

	for line in range(2, len(edl)):

		if edl[line][0] != "*" and edl[line][-1] != edl[line][-2]:  #if this doesn't have duplicate timecode for edits
			edits = {}
			dissolve_length = 0

			if edl[line][3] == "D":
				dissolve_length = int(edl[line][4])

			edits = {
				"media_type": edl[line][2][0],
				"edit_type": edl[line][3],
				"dissolve_length": dissolve_length,
				"edit_in" : edl[line][-2],
				"edit_out" : edl[line][-1]
				}

			simplified_edl.append(edits)  #  adds edits to the simplified EDL


		elif edl[line][0] == "*":  # lines with clip note
			edits = {
				"media_type" : edl[line][0],
				"media_file": edl[line][1]
			}

			simplified_edl.append(edits)  #  adds edl  to the simplified EDL 
	
	# pprint(simplified_edl)

	edl_split_video_audio(simplified_edl)	

### Detect Dropframe

In [1705]:
def detect_dropframe(edl):
	global drop_frame
     
	if edl[1][0] == "FCM:":
            if edl[1][1] == "DROP":
                drop_frame = True
            else:
                drop_frame = False


### Split Video and Audio Edits

In [1706]:
#  Looks for relevant edit information:  Drop/Non-Drop Frame, Title Name, Edits
def edl_split_video_audio(edl):
    # Clears the edit list for parsing
    global video_edits
    global audio_edits

    edl_video = []
    edl_audio = []

    for line in range(0, len(edl)):
        #  Splits audio and video edits into seperate EDLs
        if edl[line]["media_type"] == "V":
            edl_video.append(edl[line])
            if line+1 < len(edl) and edl[line+1]["media_type"] == "*": 
                edl_video.append(edl[line+1])                

        if edl[line]["media_type"] == "A":
            edl_audio.append(edl[line])
            if line+1 < len(edl) and edl[line+1]["media_type"] == "*":
                edl_audio.append(edl[line+1])


    
    # Parses the edits
    if edl_video:
        video_edits = parse_edits(edl_video)

    if edl_audio:
        audio_edits = parse_edits(edl_audio)

    # print("=========VIDEO=============")
    # pprint(edl_video)
    # print("=========AUDIO=============")
    # pprint(edl_audio)


### Parse Edits

In [1707]:
def parse_edits(edl):
    edit_list = []

    for line in range(0, len(edl)):  
        edits = {} 

        #  Audio Edits
        if line+1 < len(edl) and edl[line]["media_type"] == "A":
            #  Audio - Cut In - Cut Out 
                # (only option - no audio dissolves done in vantage  - any edits needs to be done on the media file before exporting to vantage)
            edits = {
                "type": "audio_edit",
                "dissolve_in" : 0,
                "dissolve_out" : 0,
                "edit_in" : edl[line]["edit_in"],
                "edit_out" : edl[line]["edit_out"],
                "media_file" : edl[line+1]["media_file"]            
            }
            edit_list.append(edits)

                

        # #  Video Edits
        if (edl[line]["media_type"] == "V"):

            #  DISSOLVE IN
            if (line+2 < len(edl) and
                edl[line]["edit_type"] == "D" and
                edl[line+1]["media_type"] != "*" and
                edl[line+2]["media_type"] == "*"           
                ): 

                edits = {
                    "type" : "dissolve_in",
                    "dissolve_in" : edl[line]["dissolve_length"],
                    "dissolve_out" : 0,
                    "edit_in" : edl[line]["edit_in"],
                    "edit_out" : edl[line]["edit_out"],
                    "media_file" : edl[line+2]["media_file"]            
                }

                edit_list.append(edits)

            # #  DISSOLVE OUT
            if (line+1 < len(edl) and
                edl[line]["edit_type"] == "D" and 
                edl[line+1]["media_type"] == "*"
                ): 
                     
                edits = {
                    "type" : "dissolve_out",
                    "dissolve_in" : 0,
                    "dissolve_out" : int(edl[line]["dissolve_length"]),
                    "edit_in" : edl[line]["edit_in"],
                    "edit_out" : edl[line]["edit_out"],
                    "media_file" : edl[line+1]["media_file"]            
                }
                  
                edit_list.append(edits)

            #  CUT IN CUT OUT
            if (line+1 < len(edl) and
                edl[line]["edit_type"] == "C" and 
                edl[line+1]["media_type"] == "*"
                ): 
                     
                edits = {
                    "type" : "cut_in_cut_out",
                    "dissolve_in" : 0,
                    "dissolve_out" : 0,
                    "edit_in" : edl[line]["edit_in"],
                    "edit_out" : edl[line]["edit_out"],
                    "media_file" : edl[line+1]["media_file"]            
                }
                  
                edit_list.append(edits)


    # pprint(edit_list)

    consolodated_edits = consolodate_edits(edit_list)



    # return edit_list

In [1708]:
#  parse edits v2

# def parse_edits(edl):
#     pprint(edl)
    
#     edit_list = []
#     for line in range(0, len(edl)):  
#         edits = {}     

#         # print(edl[line][0])
        
#         if edl[line][0] == "*":  

#             #  Audio - Cut In - Cut Out
#             if (edl[line-1][0] == "A"):
#                 edits = {
#                     "dissolve_in" : 0,
#                     "dissolve_out" : 0,
#                     "edit_in" : edl[line-1][3],
#                     "edit_out" : edl[line-1][4],
#                     "edit_file" : edl[line][1],            
#                 }
#                 edit_list.append(edits)
                

#             #  Video Edits
#             if (edl[line-1][0] == "V"):
#                 # print("video edit")

#                 #  DISSOLVE IN - DISSOLVE OUT
#                 if (line >= 4 and                               # add a check to make sure it isn't out of range
#                     (edl[line-1][1] == "D") and                 # is 1 line above this title a dissolve
#                     (edl[line-2][1] == edl[line][1]) and        # does 2 lines above this title match titles
#                     (edl[line-3][1] == "C") and                 # is the 3rd line above this title a cut
#                     (edl[line-3][4] == edl[line-1][3]) and      # does out point 3rd line above this title a cut and out point matches the inpoint of the dissolve 1 line aboe the title?
#                     (edl[line-4][1] == "D") and                 # is the 4th line above this title a dissolve
#                     (edl[line-4][4] == edl[line-3][3])          # does out point 4th line above a dissolve and out point matches the inpoint of the dissolve 3 line aboe the title?                
#                     ):                    

#                     print(f"Dissolve In - Dissolve out      |  In:  {edl[line-4][3]} {edl[line-4][2]} frames  Out:  {edl[line-1][4]} {edl[line-1][2]} frames  --  {edl[line][1]}")
                
#                 #  DISSOLVE IN - CUT OUT
#                 elif (line >= 2 and line+1 <= len(edl) and          # add a check to make sure it isn't out of range
#                     (edl[line-1][1] == "C") and                 # is 1 line above this title a cut
#                     (edl[line-2][1] == "D") and                 # is the 2nd line above this title a cut
#                     (edl[line-2][4] == edl[line-1][3]) and      # does out point 2nd line above this title a dissolve and out point matches the inpoint of the cut 1 line aboe the title?                
#                     (edl[line+1][1] != "D")                     #  the line 1 below this title is not a dissolve
#                         ):                    

#                     print(f"Dissolve In - Cut out           |  In:  {edl[line-2][3]} {edl[line-2][2]} frames  Out:  {edl[line-1][4]} {edl[line-1][2]} frames  --  {edl[line][1]}")


#                 #  CUT IN - DISSOLVE OUT
#                 elif (line >= 4 and                             # add a check to make sure it isn't out of range
#                     (edl[line-1][1] == "D") and                 # is 1 line above this title a dissolve
#                     (edl[line-2][1] == edl[line][1]) and        # does 2 lines above this title match titles
#                     (edl[line-3][1] == "C") and                 # is the 3rd line above this title a cut
#                     (edl[line-4][1] != "D") and
#                     (edl[line-3][4] == edl[line-1][3])       # does out point 3rd line above this title a cut and out point matches the inpoint of the dissolve 1 line aboe the title?
                       
#                     ):                    

#                     print(f"Cut In - Dissolve out           |  In:  {edl[line-3][3]} {edl[line-3][2]} frames  Out:  {edl[line-1][4]} {edl[line-1][2]} frames  --  {edl[line][1]}")

#                 #  CUT IN - CUT OUT
#                 elif (line >= 2 and line+1 <= len(edl) and     # add a check to make sure it isn't out of range
#                     (edl[line-1][1] == "C") and                  # is 1 line above this title a cut 
#                     (edl[line-2][1] !=  "D") and
#                     (edl[line+1][1] == "C") 
#                     ):                    

#                     print(f"Cut In - Cut out                |  In:  {edl[line-1][3]} {edl[line-1][2]} frames  Out:  {edl[line-1][4]} {edl[line-1][2]} frames  --  {edl[line][1]}")
                
#     return edit_list

In [1709]:
#   PARSE EDITS - v1

# def parse_edits(edl):
    
#     edit_list = []
#     for line in range(0, len(edl)):  
#         edits = {}     

#         # print(edl[line][0])
        
#         if edl[line][0] == "*":  

#             #  Audio - Cut In - Cut Out
#             if (edl[line-1][0] == "A"):
#                 edits = {
#                     "dissolve_in" : 0,
#                     "dissolve_out" : 0,
#                     "edit_in" : edl[line-1][3],
#                     "edit_out" : edl[line-1][4],
#                     "edit_file" : edl[line][1],            
#                 }
#                 edit_list.append(edits)  

#             #  Video Edits
#             if (edl[line-1][0] == "V"):
#                 print("video edit")

            # Dissolve in - Dissolve Out
            # elif  (edl[line-1][2][0] == "V"):
            #     print(edl[line-1][0])
   
                # edits = {
                #     "dissolve_in" : int(edl[line-2][4]),
                #     "dissolve_out" : int(edl[line+2][4]),
                #     "edit_in" : edl[line-2][7],
                #     "edit_out" : edl[line+2][8],
                #     "edit_file" : f"{edl[line][1]}.nbtitle",            
                # }
                # edit_list.append(edits)
            
            # # Dissolve in - Dissolve Out
            # elif  (line+3 <= len(edl) and 
            #        edl[line][1] == edl[line+3][1] and 
            #        edl[line-2][3] == "D" and 
            #        edl[line-1][3] == "C" and 
            #        edl[line+2][3] == "D"):
   
            #     edits = {
            #         "dissolve_in" : int(edl[line-2][4]),
            #         "dissolve_out" : int(edl[line+2][4]),
            #         "edit_in" : edl[line-2][7],
            #         "edit_out" : edl[line+2][8],
            #         "edit_file" : f"{edl[line][1]}.nbtitle",            
            #     }
            #     edit_list.append(edits)                
            
            # # Dissolve in - Cut out
            # elif (edl[line+1][5] != "00:00:00:00" and 
            #       edl[line-2][3] == "D" and 
            #       edl[line-1][3] == "C"):
            #     edits = {
            #         "dissolve_in" : int(edl[line-2][4]),
            #         "dissolve_out" : 0,
            #         "edit_in" : edl[line-2][7],
            #         "edit_out" : edl[line-1][7],
            #         "edit_file" : f"{edl[line][1]}.nbtitle",            
            #     }
            #     edit_list.append(edits)

            # # # Cut in - Dissolve Out
            # elif (line+3 <= len(edl) and 
            #       edl[line][1] == edl[line+3][1] and 
            #       edl[line-2][3] == "C" and edl[line-1][3] == "C" and 
            #       edl[line+2][3] == "D"):
            #     edits = {
            #         "dissolve_in" : 0,
            #         "dissolve_out" : int(edl[line+2][4]),
            #         "edit_in" : edl[line-1][7],
            #         "edit_out" : edl[line+2][8],
            #         "edit_file" : f"{edl[line][1]}.nbtitle",            
            #     }
            #     edit_list.append(edits)

            # #  Cut In - Cut Out
            # elif (line+1 <= len(edl) and 
            #       edl[line-2][3] == "C" and 
            #       edl[line-1][3] == "C" and 
            #       edl[line+1][3] == "C"):
            #     edits = {
            #         "dissolve_in" : 0,
            #         "dissolve_out" : 0,
            #         "edit_in" : edl[line-1][6],
            #         "edit_out" : edl[line-1][7],
            #         "edit_file" : f"{edl[line][1]}.nbtitle",            
            #     }
            #     edit_list.append(edits)

    # return edit_list

###  Consolodate Edits

In [1710]:
def consolodate_edits(edl):\
	pprint(edl)

### Execution 

In [1711]:
open_edl(edl_path)


# print("------Video Edits-------")
# pprint(video_edits)
# print("------Audio Edits-------")
# pprint(audio_edits)

[{'dissolve_in': 0,
  'dissolve_out': 0,
  'edit_in': '01:00:20:11',
  'edit_out': '01:00:45:05',
  'media_file': 'CUT-IN-CUT-OUT',
  'type': 'cut_in_cut_out'},
 {'dissolve_in': 10,
  'dissolve_out': 0,
  'edit_in': '01:00:47:24',
  'edit_out': '01:00:48:04',
  'media_file': 'DISSOLVE-IN-CUT-OUT',
  'type': 'dissolve_in'},
 {'dissolve_in': 0,
  'dissolve_out': 0,
  'edit_in': '01:00:48:04',
  'edit_out': '01:01:08:16',
  'media_file': 'DISSOLVE-IN-CUT-OUT',
  'type': 'cut_in_cut_out'},
 {'dissolve_in': 10,
  'dissolve_out': 0,
  'edit_in': '01:01:12:03',
  'edit_out': '01:01:12:13',
  'media_file': 'DISSOLVE-IN-DISSOLVE-OUT',
  'type': 'dissolve_in'},
 {'dissolve_in': 0,
  'dissolve_out': 0,
  'edit_in': '01:01:12:13',
  'edit_out': '01:01:31:20',
  'media_file': 'DISSOLVE-IN-DISSOLVE-OUT',
  'type': 'cut_in_cut_out'},
 {'dissolve_in': 0,
  'dissolve_out': 10,
  'edit_in': '01:01:31:20',
  'edit_out': '01:01:32:00',
  'media_file': 'DISSOLVE-IN-DISSOLVE-OUT',
  'type': 'dissolve_out'},